<a href="https://colab.research.google.com/github/rassbr/MISCELLANEOUS/blob/main/Leandro_HBSIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rodar apenas a primeira vez

In [ ]:
pip install jira-python

  Created wheel for jira-python: filename=jira_python-0.2.2-cp36-none-any.whl size=4681 sha256=6136c4469212e0a5c9a1a7136d5da7b72fd604e81a6e5c6f7f731f8cfb7ef149
  Stored in directory: /root/.cache/pip/wheels/7c/28/c3/a809f583674a8a14db759639f9e1255e5e9427c2d40f2fd201
Successfully built jira-python


In [ ]:
pip install xlsxwriter

     |████████████████████████████████| 153kB 5.4MB/s 


In [ ]:
#pip install pandas --upgrade

# Importação dos pacotes

In [ ]:
import pandas as pd
from jira.client import Client
import csv
import os
#import os.path
import time
from datetime import date
from google.colab import files
from getpass import getpass
#uploaded = files.upload()
from google.colab import drive
import csv
import datetime
import numpy as np

# Montando arquivos drive

Clique na aba ao lado e conecte o google drive caso deseje, senão pular essa etapa


In [ ]:
drive.mount('/content/drive')

In [ ]:
os.chdir('drive/My Drive')

In [ ]:
try:
    os.mkdir('Colab')
except OSError:
    print ("Creation of the directory %s failed" )
else:
    print ("Successfully created the directory %s " )
os.chdir('Colab')

In [ ]:
!ls

# Logar
se usar esse arquivo sozinho pode usar um txt ou algo para não ter que colocar sempre.

In [ ]:
login_auto = 0


HOST = 'https://jira.hbsis.com.br/'

if login_auto == 1:
  login = []
  with open('login.txt') as csvfile:
      readCSV = csv.reader(csvfile, delimiter=',')
      for row in readCSV:
        login = login + row
  USER = login[0]
  PASSWORD = login[1]

else:
  print("Coloque o seu usuário:")
  login_user = input()
  print("Coloque sua senha:")
  login_pass = getpass()
  USER = login_user
  PASSWORD = login_pass



client = Client(HOST, USER, PASSWORD) # Host must have trailing slash

Coloque o seu usuário:
rodolfo.branco
Coloque sua senha:
··········


In [ ]:
response = client.get_permissions()
print(response)

{'permissions': {'ARCHIVE_ISSUES': {'id': '-1', 'key': 'ARCHIVE_ISSUES', 'name': 'Arquivar itens', 'type': 'PROJECT', 'description': 'Permissão para arquivar itens de um projeto específico.', 'havePermission': False}, 'VIEW_WORKFLOW_READONLY': {'id': '45', 'key': 'VIEW_WORKFLOW_READONLY', 'name': 'Ver somente leitura de fluxo de trabalho', 'type': 'PROJECT', 'description': 'admin.permissions.descriptions.VIEW_WORKFLOW_READONLY', 'havePermission': True, 'deprecatedKey': True}, 'CREATE_ISSUES': {'id': '11', 'key': 'CREATE_ISSUES', 'name': 'Criar issue', 'type': 'PROJECT', 'description': 'Permissão para criar pendências.', 'havePermission': True}, 'VIEW_DEV_TOOLS': {'id': '29', 'key': 'VIEW_DEV_TOOLS', 'name': 'Ver Ferramentas de Desenvolvimento', 'type': 'PROJECT', 'description': 'Permite os usuários em um projeto de software visualizar informações relacionadas ao desenvolvimento de um item, tais como comentários, opiniões e informações de build.', 'havePermission': True}, 'BULK_CHANGE':

# Funções de base

In [ ]:
def tempo_apon(issue_fields):
    try:
        tempo_apontado = issue_fields['timetracking']['timeSpentSeconds']/3600
    except:
        tempo_apontado = 0
    return tempo_apontado

def team_fun(issue_fields):
    try:
        team = issue_fields['labels'][0]
    except:
        team = None
    return team

def team_fun_alt(issue_fields):
    try:
        name_pos = issue_fields['customfield_10714'][0].find('name')
        team = issue_fields['customfield_10714'][0][name_pos+5+9+3:name_pos+9+5+7].lower()
    except:
        team = None
    return team

def sprinting(issue_fields):
    sprints = []
    try:
        for cont in range(len(issue_fields['customfield_10714'])):
            str0 = issue_fields['customfield_10714'][cont]
            #sprint = issue_fields['customfield_10714'][cont][name_pos+5+7:name_pos+9+5]
            #sprints = sprints + [sprint]
            pos = str0.find('name')
            str1 = str0[pos:].split(',')[0]
            sprint = [int(s) for s in str1.split() if s.isdigit()]
            sprints = sprints + sprint
            sprints[0]
    except:
        sprints = [999]
        pass
    return sprints

def sprinting_alt(issue_fields):
    #sprints = []
    try:
        #for cont in range(len(issue_fields['customfield_10714'])):
        #    name_pos = issue_fields['customfield_10714'][cont].find('name')
        #    sprint = issue_fields['customfield_10714'][cont][name_pos+5+7:name_pos+9+5]
        #     sprints = sprints + [int(sprint)]
        pos = len(issue_fields['customfield_10714'])-1
        name_pos = issue_fields['customfield_10714'][pos].find('name')
        sprint = issue_fields['customfield_10714'][pos][name_pos+5+7:name_pos+9+5]
        sprints = int(sprint)
    except:
        sprints = None
        #pass
    return sprints[-2]

def blocking(issue_fields):
    try:
        if issue_fields['priority']['name'] == 'Blocker':
            blocked = 1
        else:
            blocked = 0
    except:
        blocked = 0
    return blocked

def estimates(issue_fields):
    try:
        answer = issue_fields['timetracking']['originalEstimateSeconds']/3600
        
    except:
        answer = 0
    return answer

def project(n):
    if n == 1:
        return 'B2BG-'
    elif n == 2:
        return 'B2BMON-'
    elif n == 3:
        return 'B2BACC-'
    elif n == 4:
        return 'B2BPL-'
    elif n == 5:
        return 'CRM-'
    elif n == 6:
        return 'B2BSUP-'
    elif n == 7:
        return 'RDA-'
    else:
        return 'B2A-'
    
def status_verify(string):   
    if string == 'Concluída' or string == 'Cancelada' or string == 'Cancelado' or string == 'A Faturar':
        return False
    elif string == "Faturado"  :
        return False
    else:
        return True
    
    
def reprocess(issue,data_old):
    try:
        issue_response = client.get_issue(issue)
        issue_fields = issue_response['fields']
        nome = issue_fields['summary']

        #functions
        tempo_apontado = tempo_apon(issue_fields)
        tempo_estimado = estimates(issue_fields)
        status = issue_fields['status']['name']
        blocked = blocking(issue_fields)
        team = team_fun_alt(issue_fields)
        sprints = sprinting(issue_fields)
        sprint0 = sprints[0]
        sprint1 = sprints[-1]
        tipo_card = issue_fields['issuetype']['name']
        data_at = date.today()
        status_age = data_at - data_old
        a = [issue, nome, tipo_card, tempo_apontado,tempo_estimado, status, blocked, team,status_age,data_at, sprint0,sprint1]
        return a
        #counter = 0
        #i_pos += 1
    except:
        #print ('Fail: ' + str(i))
        #counter += 1
        #if counter >  50:
        #    break
        #pass
        return None
    
    
def sprintdate(issue_fields,esc):
    if esc == 1:
        esc = -1
    
    try:
        ref = ['startDate=','endDate=','completeDate=']
        lista = []
        for i in ref:
            str0 = issue_fields['customfield_10714']
            pos = str0[esc].find(i)
            str1 = str0[esc][pos:].split(',')[0]
            str2 = str1.split('=')[1]
            lista = lista + [datatonormal(str2)]
    except:
        return None
    return lista

def cardcreationdate(issue_fields):
    try:
        
        return datatonormal(issue_fields['created'])
    except:
        return None

def cardresolutiondate(issue_fields):
    try:
        return datatonormal(issue_fields['resolutiondate'])
    except:
        return None
    
    
def datatonormal(str2):
    return datetime.datetime.strptime(str2.split('T')[0], '%Y-%m-%d')

In [ ]:
projects_num = 8

j = 0
posi = 0
column_names = ["PROJETO", "ID", "NOME", "TIPO","TEMPO_APONTADO", "TEMPO_ESTIMADO","STATUS", "BLOQUEIO","TIME","DIAS_DESDE_ATUALIZAÇÃO","DATA_ATUALIZAÇÃO","DATA_CRIAÇÃO",'DATA_RESOLUÇÃO','DATA_SPRINT_INICIO','DATA_SPRINT_FIM',"SPRINT_INICIO", "SPRINT_FIM"]
df = pd.DataFrame(columns = column_names)

column_names1 = ["PROJETO","SPRINT", "DATA_INICIO", "DATA_FIM", "DATA_COMPLETA"]
df1 = pd.DataFrame(columns = column_names1)
    
for ind in range(projects_num):
    print(project(ind))
    ini = 0



    counter = 0 
    for i in range(ini+1, ini+10000):
        if i % 10 ==0:
            print (i)
        issue = project(ind)+ str(i)
        #issue = 'CRM-2735'
        try:
            issue_response = client.get_issue(issue)
            issue_fields = issue_response['fields']
            nome = issue_fields['summary']

            #functions
            tempo_apontado = tempo_apon(issue_fields)
            tempo_estimado = estimates(issue_fields)
            status = issue_fields['status']['name']
            blocked = blocking(issue_fields)
            team = team_fun_alt(issue_fields)
            sprints = sprinting(issue_fields)
            sprint0 = sprints[0]
            sprint1 = sprints[-1]

            result  = sprintdate(issue_fields,0)
            if result != None:
                inicio, descarte, descarte = result
            else:
                inicio = None
              
            
            result  = sprintdate(issue_fields,1)
            if result != None:
                descarte, fim, descarte = result
            else:
                fim = None
            
            criacao = cardcreationdate(issue_fields)
            resolucao = cardresolutiondate(issue_fields)

            tipo_card = issue_fields['issuetype']['name']
            data_at = date.today()
            status_age = 0
            #df.loc[i] = [issue, nome, tipo_card, tempo_apontado,tempo_estimado, status, blocked, team,status_age,data_at, sprint0,sprint1]
            df.loc[posi] = [project(ind),issue, nome, tipo_card, tempo_apontado,tempo_estimado, status, blocked, team,status_age,data_at, criacao, resolucao, inicio, fim, sprint0,sprint1]
            counter = 0
            posi += 1
            
            if sprint0 not in sprint_control:
                sprint_control = sprint_control + [sprint0]
                inicio,fim,completa = sprintdate(issue_fields,0)
                df1.loc[j] = [project(ind), sprint0, inicio, fim, completa]
                j += 1


            #i_pos += 1
        except:
            #print ('Fail: ' + str(i))
            counter += 1
            if counter >  50:
                break
            pass
    print('xau')
    
    
    df.to_parquet('Leandro_file')
    df.to_pickle('Leandro_file'+'.pkl')
    
#print(df.head())

print("DONE!")
#df.to_pickle('AllJira_alt1.pkl')
df.to_excel('Leandro_file'+'.xls')
end = time.time()

total_time = end-ini
print(total_time)

B2A-
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
xau
B2BG-
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
xau
B2BMON-
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
xau
B2BACC-
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
25

In [ ]:
ind = 1
counter = 0
df = pd.read_parquet( 'Leandro_file' )

column_names2 = ["PROJETO","ID","TH_SEMANA", "LT_SEMANA", "CT", "LT", "ANO_RESOLUÇÃO", "Ano-Sem", "SEMANA","SEMANA-DIA","TIPO"]
df2 = pd.DataFrame(columns = column_names2)
for i in df.index:
    try:
        issue = df.loc[i]
        issue_id = issue["ID"]
        status = issue['STATUS']
        issue_date = issue["DATA_ATUALIZAÇÃO"]

        TH_sem = df.loc[i]['DATA_RESOLUÇÃO'].isocalendar()[1]
        LT_sem = df.loc[i]['DATA_CRIAÇÃO'].isocalendar()[1]
        CT = (df.loc[i]['DATA_RESOLUÇÃO'] - df.loc[i]['DATA_SPRINT_INICIO']).days
        LT = (df.loc[i]['DATA_RESOLUÇÃO'] - df.loc[i]['DATA_CRIAÇÃO']).days
        
        fimsprint_sem = df.loc[i]['DATA_SPRINT_FIM'].isocalendar()[1]
        semana = df.loc[i]['DATA_SPRINT_FIM'].isocalendar()
        semana_dia = df.loc[i]['DATA_SPRINT_FIM']
        
        ANO = df.loc[i]['DATA_RESOLUÇÃO'].isocalendar()[0]
        TIPO = df.loc[i]["TIPO"]
        PROJETO = df.loc[i]["PROJETO"]
        
        ano_sem = str(ANO) + '-' + str(fimsprint_sem)
        df2.loc[i] = [PROJETO, issue_id, TH_sem, LT_sem, CT, LT, ANO, ano_sem, semana, semana_dia,TIPO ]
    except:
        pass
    
column_names3 = ["PROJETO","CT_85","CT_15","LT_85", "LT_15"]
df3 = pd.DataFrame(columns = column_names3)    
for i in range(5):
    #CT_85 = np.percentile(df2.where (df2["PROJETO"] == project(i)).where(df2["TIPO"]!="Épico").dropna()['CT'],85)
    #CT_15 = np.percentile(df2.where(df2["PROJETO"] == project(i) ).where(df2["TIPO"]!="Épico").dropna()['CT'],15)
    #LT_85 = np.percentile(df2.where(df2["PROJETO"] == project(i) ).where(df2["TIPO"]!="Épico").dropna()['LT'],85)
    #LT_15 = np.percentile(df2.where(df2["PROJETO"] == project(i) ).where(df2["TIPO"]!="Épico").dropna()['LT'],15)
    CT_85 = np.percentile(df2.where (df2["PROJETO"] == project(i)).dropna()['CT'],85)
    CT_15 = np.percentile(df2.where(df2["PROJETO"] == project(i) ).dropna()['CT'],15)
    LT_85 = np.percentile(df2.where(df2["PROJETO"] == project(i) ).dropna()['LT'],85)
    LT_15 = np.percentile(df2.where(df2["PROJETO"] == project(i) ).dropna()['LT'],15)
    df3.loc[i] = [project(i), CT_85,CT_15,LT_85,LT_15]

In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('DadosMetricas.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
df.to_excel(writer, sheet_name='DADOS')
#df1.to_excel(writer, sheet_name='sprints')    
df2.to_excel(writer, sheet_name='Tratados')    
df3.to_excel(writer, sheet_name='Resumo')    

# Write each dataframe to a different worksheet.
#dt.to_excel(writer, sheet_name='Sheet1')


# Close the Pandas Excel writer and output the Excel file.
writer.save()


In [ ]:
from google.colab import files
files.download('DadosMetricas.xlsx') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>